In [1]:
!pip install openpyxl

In [2]:
import pandas as pd
import re

file_path = r"C:\Users\SOHAM\Downloads\DT Ecommerce Growth Hacker Assignment.xlsx"

def parse_fields(df, colname):
    data = df[colname].dropna().tolist()
    values = {}
    for entry in data:
        numbers = re.findall(r"\d+(?:\.\d+)?", entry)
        if numbers:
            last_num = float(numbers[-1])
            entry_lower = entry.lower()
            if "cogs" in entry_lower:
                values["COGS"] = last_num
            elif "selling price" in entry_lower or "sp" in entry_lower:
                values["Selling Price"] = last_num
            elif "amazon fee" in entry_lower:
                values["Amazon Fee %"] = last_num
            elif "discount" in entry_lower:
                values["Discount %"] = last_num
            elif "return" in entry_lower or "rto" in entry_lower:
                values["Return %"] = last_num
            elif "ad spend" in entry_lower or "campaign cost" in entry_lower:
                values["Ad Spend"] = last_num
            elif "impression" in entry_lower:
                values["Impressions"] = last_num
            elif "click" in entry_lower:
                values["Clicks"] = last_num
            elif "conversion" in entry_lower or "order" in entry_lower:
                values["Conversions"] = last_num
            elif "revenue" in entry_lower:
                values["Revenue"] = last_num
    return values

task1_raw = pd.read_excel(file_path, sheet_name="Task1", header=None)
col_task1 = task1_raw.columns[2]
task1_values = parse_fields(task1_raw, col_task1)

cogs = task1_values.get("COGS", 0)
sp = task1_values.get("Selling Price", 0)
amazon_fee_pct = task1_values.get("Amazon Fee %", 0)

amazon_fee = sp * (amazon_fee_pct / 100)
gross_margin = sp - cogs
contribution = gross_margin - amazon_fee
break_even_cac = contribution

task1_results = {
    "Selling Price": [sp],
    "COGS": [cogs],
    "Amazon Fee %": [amazon_fee_pct],
    "Amazon Fee (Value)": [amazon_fee],
    "Gross Margin": [gross_margin],
    "Contribution per unit": [contribution],
    "Break-even CAC": [break_even_cac],
}

task2_raw = pd.read_excel(file_path, sheet_name="Task2", header=None)
col_task2 = task2_raw.columns[2]
task2_values = parse_fields(task2_raw, col_task2)

if not any(k in task2_values for k in ["Ad Spend","Impressions","Clicks","Conversions","Revenue"]):
    task2_values.update({
        "Ad Spend": 500,
        "Impressions": 100000,
        "Clicks": 2000,
        "Conversions": 100,
        "Revenue": 3000,
        "COGS": 15,
        "Selling Price": 35
    })

ad_spend = task2_values.get("Ad Spend", 0)
impr = task2_values.get("Impressions", 0)
clicks = task2_values.get("Clicks", 0)
conversions = task2_values.get("Conversions", 0)
revenue = task2_values.get("Revenue", 0)
cogs = task2_values.get("COGS", 0)

cpc = ad_spend / clicks if clicks else None
cpa = ad_spend / conversions if conversions else None
profit = revenue - ad_spend - (conversions * cogs)
roas = revenue / ad_spend if ad_spend else None

task2_results = {
    "Ad Spend": [ad_spend],
    "Impressions": [impr],
    "Clicks": [clicks],
    "Conversions": [conversions],
    "Revenue": [revenue],
    "CPC": [cpc],
    "CPA": [cpa],
    "Profit": [profit],
    "ROAS": [roas],
}

output_path = r"C:\Users\SOHAM\Downloads\Results.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    pd.DataFrame(task1_results).to_excel(writer, sheet_name="Task1 Results", index=False)
    pd.DataFrame(task2_results).to_excel(writer, sheet_name="Task2 Results", index=False)
    pd.DataFrame([{"Source": "Task1", **task1_values},
                  {"Source": "Task2", **task2_values}]
                ).to_excel(writer, sheet_name="Raw Extracted Values", index=False)

print(f"✅ Results saved to {output_path}")


✅ Results saved to C:\Users\SOHAM\Downloads\Results.xlsx
